In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('$100')

['$', '100']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words))
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['hi'],['how','hi'])

array([0., 1.])

In [8]:
data = pd.read_csv('./data.csv')

In [9]:
X = data['text']
y = data['target']

In [10]:
all_words = []
all_data = []
tags = []

In [11]:
for X_batch,y_bacth in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    all_words.extend(new_X)
    all_data.append([new_X,y_bacth])
    tags.append(y_bacth)

7613it [00:02, 3223.69it/s]


In [12]:
np.random.shuffle(all_words)

In [13]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [14]:
np.random.shuffle(all_words)

In [15]:
X = []
y = []

In [16]:
for X_batch,y_batch in tqdm(all_data):
    X.append(bag_of_words(X_batch,all_words))
    y.append(tags.index(y_batch))

100%|██████████████████████████████████████| 7613/7613 [00:17<00:00, 436.49it/s]


In [17]:
from sklearn.model_selection import *

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [19]:
device = 'cuda'

In [20]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).to(device).float()

In [21]:
tags

[0, 1]

In [22]:
class Model(Module):
    def __init__(self,hidden):
        super().__init__()
        self.iters = 12
        self.activation = ReLU()
        self.linear1 = Linear(len(all_words),hidden)
        self.linear2 = Linear(hidden,hidden)
        self.output = Linear(hidden,1)
    
    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(self.iters):
            preds = self.activation(self.linear2(preds))
        preds = self.output(preds)
        return preds

In [23]:
model = Model(512).to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 1000
batch_size = 32

In [24]:
PROJECT_NAME = 'NLP-with-Disaster-Tweets-Clf-V8'

In [25]:
import wandb

In [26]:
def get_loss(model,X,y,criterion):
    return criterion(model(X).view(-1,1),y.view(-1,1)).item()

In [27]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [ ]:
torch.cuda.empty_cache()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    torch.cuda.empty_cache()
    for idx in range(0,len(X_train),batch_size):
        torch.cuda.empty_cache()
        X_batch = X_train[idx:idx+batch_size]
        y_batch = y_train[idx:idx+batch_size]
        preds = model(X_batch)
        loss = criterion(preds.view(-1,1),y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})    
wandb.finish()
torch.cuda.empty_cache()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


 11%|████▏                                   | 106/1000 [02:31<21:09,  1.42s/it]